# FABRIC Example: Get Map of Created Resources


## Configure the Environment

In [10]:
import os
from fabrictestbed.slice_manager import SliceManager, Status
import json

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")
print(os.environ['FABRIC_CREDMGR_HOST'])

#Configure SSH Key
ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read().strip()

CM Host: beta-2.fabric-testbed.net Orchestrator Host: beta-7.fabric-testbed.net
beta-2.fabric-testbed.net


### Save the Initial Refresh Token

In [11]:
#Retrieve or set the refresh token (exprires 24 hours after login)
not_found=False
fabric_refresh_token=None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
print("Fabric Refresh Token {}".format(fabric_refresh_token))
print("CILOGON_REFRESH_TOKEN environment variable: {}".format(os.environ['CILOGON_REFRESH_TOKEN']))

Fabric Refresh Token NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTMYZXGMYTAMBVGAZDKY3CMJTDCMZYHEZGMMLDGEYWMZLGHAZTKP3UPFYGKPLSMVTHEZLTNBKG623FNYTHI4Z5GE3DEOBRHA4DOMRXGU2TAJTWMVZHG2LPNY6XMMROGATGY2LGMV2GS3LFHU4DMNBQGAYDAMA
CILOGON_REFRESH_TOKEN environment variable: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTIZBYGY2TIMDCMIYWINTFGJTDAN3DGFSTKZRVMQZTEMZQMJTDQOJ7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRYGE4DIOBTGA4TMNRGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ


## Create and Initialize Slice Manager Object
Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters in the refresh call below.

In [12]:
slice_manager = SliceManager(oc_host = orchestrator_host, cm_host = credmgr_host, project_name = 'all', scope = 'all')
slice_manager.initialize()

### Refresh the ID Token

ID Tokens expire one hour after refresh. 

In [18]:
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
print()
print("New Refresh Token: {}".format(fabric_refresh_token))
print()
print("Stored new Refresh Token")
%store fabric_refresh_token


New Refresh Token: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTGZJQGVSDSY3BGY2WMYTFMIYDCMBYMZSTCZRTMI4TSZRTGU2GIYJ7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRYGE4DSMZWG43DKNJGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ

Stored new Refresh Token
Stored 'fabric_refresh_token' (str)


### Query Slices

In [14]:
status, slices = slice_manager.slices(state = "All")

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slices {}".format(slices))
else:
    print(f"Failure: {slices}")

Response Status Status.OK
Slices [{
    "graph_id": "dd31e48c-294c-40a5-8f65-d0fa67b5e29d",
    "lease_end": "2021-07-27 21:15:31",
    "slice_id": "70fbefce-f374-4540-8d74-e85e4f1356f7",
    "slice_name": "Slice-l2bridge-ded-tag",
    "slice_state": "Dead"
}, {
    "graph_id": "f5fcaddb-78f1-4c67-9053-75355e29fcb9",
    "lease_end": "2021-07-29 17:55:03",
    "slice_id": "4d09c6a1-1097-467e-b858-e543feee6d9a",
    "slice_name": "Slice-l2bridge-ded-tag",
    "slice_state": "Dead"
}, {
    "graph_id": "9fc6a698-ffbc-4d2a-9a08-58237f467322",
    "lease_end": "2021-07-29 20:21:44",
    "slice_id": "96fbf26a-d0c7-49a4-ac04-8c38c13f2444",
    "slice_name": "Slice-l2bridge-ded-tag",
    "slice_state": "Closing"
}, {
    "graph_id": "38b0f0c1-d5f9-4bd8-81d1-2417f90e23fe",
    "lease_end": "2021-07-29 20:35:35",
    "slice_id": "73f9efcd-5e97-41aa-9d8f-f66a9895b27b",
    "slice_name": "Slice-sts-ded",
    "slice_state": "Dead"
}, {
    "graph_id": "e26d163c-a8f2-4207-b8a9-28f269b33034",
    "l

### Get ID of Most Recent Slice, Query Slivers From Slice

In [15]:
slice_id = slices[-1].slice_id
status, slivers = slice_manager.slivers(slice_id = slice_id)

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slivers {}".format(slivers))
else:
    print(f"Failure: {slivers}")

Response Status Status.OK
Slivers [{
    "graph_node_id": "97336be3-62ef-4a64-ac02-9444eff9e3c8",
    "join_state": "NoJoin",
    "lease_end": "2021-07-30 14:42:39",
    "name": "sts1",
    "pending_state": "None_",
    "reservation_id": "a4bba7fb-d0c3-4276-9086-6e30437c931a",
    "reservation_state": "Closed",
    "resource_type": "L2STS",
    "slice_id": "e11a82ae-bcf1-44a9-86bf-ab4980119385"
}, {
    "allocated_capacities": "{\"core\": 2, \"disk\": 10, \"ram\": 8}",
    "allocated_labels": "{\"instance_parent\": \"uky-w2.fabric-testbed.net\"}",
    "capacities": "{\"core\": 2, \"disk\": 10, \"ram\": 6}",
    "capacity_hints": "{\"instance_type\": \"fabric.c2.m8.d10\"}",
    "graph_node_id": "b784b3de-acbc-4bc7-b8c0-dca55196ece0",
    "join_state": "NoJoin",
    "lease_end": "2021-07-30 14:42:39",
    "name": "n1",
    "pending_state": "None_",
    "reservation_id": "453edf87-34fa-4b93-bbad-a34fb58fb03e",
    "reservation_state": "Closed",
    "resource_type": "VM",
    "site": "UKY"

### Create and Save Map

In [16]:
site_dict = {'UKY': (38.03154264461649, -84.50343326019218), 'RENC': (35.93996154497003, -79.01809264307425), 'LBNL': (38.94977311613871, -122.61289106281062)}
labels = {}

for sliver in slivers:
    status, sliver_status = slice_manager.sliver_status(slice_id = slice_id, sliver_id = sliver.reservation_id)
    if sliver_status.resource_type == "VM":
        name = sliver_status.name
        site = sliver_status.site
        capacities = sliver_status.capacities
        info = (site, capacities, site_dict[site])
        labels[name] = info

import folium
import branca
from folium.plugins import MarkerCluster
from ipywidgets import Text, HTML, HBox, Layout, VBox

base = folium.Map(location = (38.12480976137421, -95.7129), zoom_start = 4.2)

locations = []
popups = []
icons = []


for label in labels:
    locations.append(site_dict[labels[label][0]])
    popups.append(folium.Popup(branca.element.IFrame(html = f'<strong>Resource: {label}</strong><br>Site: {labels[label][0]}<br>Capacities: {labels[label][1]}', width = 300, height = 60), max_width = 300))
    icons.append(folium.Icon(color = 'green', icon = 'server', prefix = 'fa'))
    
base.add_child(MarkerCluster(locations = locations, popups = popups, icons = icons))

base.save('map.html')

base